In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

id = ''
password = ''

def search_biz(keyword, area, input_addr):
    time.sleep(random.uniform(2,3))
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36"
    }
    url = f"https://bizno.net/?area={area}&query={keyword}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    posts = soup.select("div.single-post")
    for post in posts:
        a_tag = post.select_one("a[href^='/article/']")
        if a_tag:
            link = "https://bizno.net"+ a_tag["href"]
        addr = post.select_one("p")
        
        addrchg = str(addr).strip()

        # 정규표현식으로 '주소 :' 다음에 나오는 도로명 주소만 추출
        match = re.search(r"주소\s*:\s*([^\n\.]+)", addrchg)
        if match:
            address = match.group(1).strip()
            clean_address = extract_up_to_road_or_dong(address)
        else:
            return 0
        if input_addr == clean_address:
            return link
    return 0
    
def get_biz_number(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    for row in soup.select("table.table_guide01 tr"):
        th = row.select_one("th")
        td = row.select_one("td")
        if th and "사업자등록번호" in th.text:
            return td.get_text(strip=True)
    return None


def extract_up_to_road_or_dong(full_address):
    # 괄호 제거
    address = re.sub(r"\([^)]*\)", "", full_address)

    if address.startswith("전북특별자치도"):
        address = address.replace("전북특별자치도", "전라북도", 1)
    
    if "강원특별자치도" in address:
        address = address.replace("강원특별자치도", "강원도", 1)

    # 시/도 + 시/군/구까지 추출
    prefix_match = re.match(r"^(.*?(시|도)\s+.*?(시|군|구))\s+", address)
    if not prefix_match:
        return None  # 주소 형식이 예상과 다르면 None

    prefix = prefix_match.group(1)

    rest = address[prefix_match.end():]

    # 도로명: 대로, 로
    road_match = re.search(r"([가-힣0-9]+?(대로|로))", rest)
    if road_match:
        return prefix + ' ' + road_match.group(1)

    # '동'이나 '번길' '길'이 있는 경우
    dong_match = re.search(r"([가-힣0-9]+?(동|번길|길))", rest)
    if dong_match:
        return prefix + ' ' + dong_match.group(1)

    return prefix  # fallback: 시도/시군구만
            
def search_fin(disno):
        time.sleep(5)
        options = Options()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        while True:
            try:
                # 로그인
                driver = webdriver.Chrome(options=options)
                driver.implicitly_wait(2)    
                driver.get('https://sminfo.mss.go.kr/cm/sv/CSV001R0.do')
                #driver.find_element(By.XPATH, '//*[@id="wrap"]/div[1]/div[2]/ul/li[2]/a').click()
                Enter_id = driver.find_element( By.XPATH,'//*[@id="login_id"]')
                Enter_id.send_keys(f'{id}')

                Enter_password=driver.find_element(By.XPATH,'//*[@id="login_password"]')
                Enter_password.send_keys(f'{password}')
                time.sleep(random.uniform(1,2))
                driver.find_element(By.XPATH,'//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button').click()
                driver.implicitly_wait(2)
                driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/a').click()
                driver.find_element(By.XPATH,'//*[@id="wrap"]/div[2]/div/ul/li[1]/ul/li[1]/a').click()
                driver.find_element(By.XPATH, '//*[@id="cmQueryOptionCombo"]').click()
            except:
                driver.quit()
                continue
            else:
                break
        
        #사업자 번호를 통해 검색
        try:
            driver.find_element(By.XPATH, '//*[@id="cmQueryOptionCombo"]/option[4]').click()
            Endter_keyword = driver.find_element(By.XPATH, '//*[@id="searchTxt"]')
            Endter_keyword.send_keys(f"{disno}")
            driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/form[4]/div/div[4]/button[1]').click()
            driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr/td[1]/a').click()
        except:# 사업자 번호가 없는경우
            driver.quit()
            return None
        try:
            table = driver.find_element(By.XPATH, "//table[caption[contains(text(), '매출현황')]]")
            rows = table.find_elements(By.XPATH, ".//tbody/tr")

            if not rows:
                driver.quit()
                return None
        
            result_list = []
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) == 7:
                    result_list.append({
                        "회사명" : keyword.strip(),
                        "결산년도": cells[0].text.strip(),
                        "총자산": cells[1].text.strip(),
                        "자본금": cells[2].text.strip(),
                        "자본총계": cells[3].text.strip(),
                        "매출액": cells[4].text.strip(),
                        "영업이익": cells[5].text.strip(),
                        "당기순이익": cells[6].text.strip(),
                    })

            driver.quit()
            return result_list

        except:
            driver.quit()
            return None
        

full_to_short_map = {
    '서울특별시': '서울',
    '경기도': '경기',
    '충청남도': '충남',
    '충청북도': '충북',
    '경상북도': '경북',
    '경상남도': '경남',
    '전라북도': '전북',
    '전라남도': '전남',
    '전북특별자치도' : '전북',
    '강원특별자치도': '강원',
    '강원도': '강원',
    '제주특별자치도': '제주',
    '제주도': '제주',
    '인천광역시': '인천',
    '부산광역시': '부산',
    '울산광역시': '울산',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '광주광역시': '광주',
    '세종특별자치시': '세종'
}

results = []
file_path = ""
df1 = pd.read_csv(file_path, encoding="utf-8-sig")
results = []
count = 0
for i in range(0,10): 
    try:
        area = full_to_short_map.get(df1['시도명'][i])
        keyword = df1['회사명'][i]
        compare_adress = extract_up_to_road_or_dong(df1['공장주소'][i])
        if compare_adress == None:
            continue
        link = search_biz(keyword,area,compare_adress)
        fin_list = []
        if link != 0:
            biznochg = re.search(r'\d+', link).group()
            fin_list = search_fin(biznochg)
            if fin_list != None:
                for k in range(len(fin_list)):
                    results.append(fin_list[k])
                count += 1
                print(f"추출 갯수: {count}")
        print(f"크롤링한 갯수:{i}")
    except:
        continue

if results:
    df = pd.DataFrame(results)
    save_path = ""
    df.to_csv(save_path, index=False, encoding="utf-8-sig")